# Interface
LCEL为了尽可能简化用户创建自定义链的过程，实现了一个**Runnable**的协议。大多数组件都已实现了Runnable协议。这是一个标准接口，可以轻松地定义自定义链并以标准方式调用它们。标准接口包括：
- stream: 返回响应的数据块
- invoke: 对输入进行链式处理
- batch: 对输入列表进行批量链式调用

这些方法也有对应的异步方法：
- astream: 异步返回响应的数据块
- ainvoke: 异步对输入进行链式处理
- abatch: 异步对输入列表进行批量链式调用
- astream_log: 除了最终的响应之外，流式返回中间步骤的发生
  
下面是各个组件的输入与输出:

| 组件 | 输入类型 | 输出类型 |
|:-------:|:-------:|:-------:|
| Prompt | string，聊天消息列表 或 PromptValue | ChatMessage |
| LLM | string，聊天消息列表 或 PromptValue	 | String |
| OutputParser | LLM 或 ChatModel 输出	 | 取决于Parser |
| Retriever | 	string	 | Documents列表 |
| Tool | string或dictionary，取决于Tool	 | 取决于Tool |

所有可运行的都公开了输入和输出模式，以便检查输入和输出： 
- input_schema: 从可运行结构自动生成的输入Pydantic模型 
- output_schema: 从可运行结构自动生成的输出Pydantic模型

让我们来看看这些方法。为此，我们将创建一个超级简单的PromptTemplate + ChatModel链。

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

model = ChatOpenAI(
    
)
prompt = ChatPromptTemplate.from_template("我想去{topic}旅行，我想知道这个地方有什么好玩的")
chain = prompt | model

## Input Schema

一个描述Runnable接受的输入的说明。这是一个根据任何Runnable结构动态生成的Pydantic模型。您可以调用.schema()来获取JSONSchema表示。

In [3]:
# 链的输入模式是其第一部分，提示的输入模式。
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [4]:
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [5]:
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'A Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'str

## Output Schema

一个Runnable生成的输出描述。这是一个根据任何Runnable结构动态生成的Pydantic模型。您可以调用.schema()来获取JSONSchema表示形式。

In [6]:
# 链的输出模式是其最后一部分的输出模式，即一个ChatModel，在这种情况下，它会输出一个ChatMessage chain.output_schema.schema()
chain.output_schema.schema()

{'title': 'ChatOpenAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'AIMessage': {'title': 'AIMessage',
   'description': 'A Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'},
      {'type': 'array',
       'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'ai',
     'enum': ['ai'],
     'type': 'string'},
    'example': {'title': 'Example', 'default': False, 'type': 'boolean'}},
   'required': ['content']},
  'HumanMessage': {'title': 'HumanMessage',
   'description': 'A Message from a human.',
   'type': 'object',
   'properties': {'conten

## Stream

In [9]:
for s in chain.stream({"topic": "云南"}):
    print(s.content, end="", flush=True)

云南是一个充满自然美景和多元文化的地方，有很多好玩的景点和活动。以下是一些云南旅行的建议：

1. 丽江古城：这是一个保存完好的古老城市，有许多古建筑和石板街道，可以感受到古老的纳西文化。

2. 玉龙雪山：这是云南著名的雪山景点，可以乘坐缆车到达山顶，欣赏雪山风光。

3. 三江并流：这是三条不同颜色的河流在一起的地方，非常壮观。

4. 香格里拉：这是一个藏区城市，有着浓厚的藏族文化，可以参观松赞林寺和观赏美丽的自然风光。

5. 云南民族村寨：云南有许多不同的少数民族，他们各自有着独特的文化和传统，可以到各个民族村寨了解他们的生活和风俗。

6. 东巴文化：云南有着独特的东巴文化，可以到东巴村寨感受这种独特的宗教文化。

此外，云南还有很多美丽的自然风光，如石林、虎跳峡、泸沽湖等，可以进行徒步、骑行、摄影等各种户外活动。总之，云南是一个充满魅力的旅行目的地，有着丰富的文化和自然资源，适合各种类型的旅行者。

## Invoke

In [8]:
chain.invoke({"topic": "上海"})

AIMessage(content='上海是中国的一座现代化大都市，有很多有趣的地方可以参观和体验。以下是一些上海旅行的建议：\n\n1. 东方明珠塔：这座高耸的地标建筑是上海的象征之一，可以乘坐观光电梯，从塔顶俯瞰整个城市的美景。\n\n2. 外滩：这是上海最古老的地区之一，有许多欧式建筑和历史悠久的码头，可以欣赏黄浦江的美景。\n\n3. 田子坊：这是一个著名的文艺区，有许多小巷和文艺小店，适合漫步和购物。\n\n4. 上海迪士尼乐园：如果你喜欢游乐园和迪士尼的童话世界，这里是一个很好的选择。\n\n5. 豫园：这是一个古老的园林，有着精美的园林景观和传统的建筑，可以感受中国古代园林的气息。\n\n除此之外，上海还有许多美食、购物和夜生活等等等等，可以根据自己的兴趣和时间安排来选择适合自己的行程。希望你能在上海度过愉快的旅行！')

## Batch

In [10]:
chain.batch([{"topic": "云南"}, {"topic": "上海"}])

[AIMessage(content='云南是一个充满魅力的旅游目的地，有许多令人兴奋的活动和景点。以下是一些你可能会喜欢的：\n\n1. 丽江古城：这是一个保留了许多古代建筑和文化的古城，非常适合步行游览和探索。\n\n2. 玉龙雪山：这是云南最著名的山脉之一，可以乘坐缆车到达山顶，欣赏壮丽的景色。\n\n3. 丽江樱花节：每年春天，丽江都会举办樱花节，吸引大量游客前来观赏樱花盛开的美景。\n\n4. 丽江古城周边的茶马古道：可以选择骑马或者徒步沿着茶马古道，探索当地的自然风光和文化遗产。\n\n5. 云南民族村落：云南有许多不同的少数民族，他们的村落和传统文化都很值得一游。\n\n总之，云南有很多值得一游的地方，无论你是喜欢自然风光还是文化遗产，都能在云南找到适合自己的旅行体验。'),
 AIMessage(content='上海是中国最繁华的城市之一，有很多有趣的地方可以游览。一些著名的景点包括：\n\n1. 外滩：这是上海的标志性景点，可以欣赏到黄浦江的美景，还有许多历史建筑和现代摩天大楼。\n\n2. 东方明珠塔：这是上海的地标性建筑，是一座高塔，可以乘坐观光电梯俯瞰整个城市。\n\n3. 田子坊：这是一个具有上海特色的老街区，有许多小吃、手工艺品和艺术品可以购买。\n\n4. 上海迪士尼乐园：如果你喜欢游乐园，可以去上海迪士尼乐园体验一下。\n\n5. 城隍庙：这是上海的一座古老寺庙，围绕着一条小吃街，可以品尝到各种上海特色小吃。\n\n此外，上海还有许多博物馆、购物中心、美食街和夜景可以探索。希望你能在上海玩得开心！')]

您可以使用**max_concurrency**参数来设置并发请求的数量。

In [11]:
chain.batch([{"topic": "云南"}, {"topic": "上海"}], config={"max_concurrency": 5})

[AIMessage(content='云南是中国西南部的一个省份，拥有丰富的自然风光和文化遗产。如果你想去云南旅行，可以考虑以下几个地方：\n\n1. 丽江古城：丽江古城是世界文化遗产，以其独特的纳西族文化和古老的建筑而闻名。你可以在这里漫步古街，品尝纳西美食，还可以去玉龙雪山观赏壮丽的雪景。\n\n2. 香格里拉：香格里拉是云南西北部的一个藏区城市，被誉为“世外桃源”。你可以在这里感受浓厚的藏族文化氛围，去普达措国家公园欣赏迷人的自然风光。\n\n3. 云南大理：大理是云南的另一个古城，有着悠久的历史和宁静的湖泊，你可以在这里骑行环洱海、游览古城、欣赏苍山洱海的美景。\n\n4. 云南石林：石林是中国四大石林之一，以其奇特的石灰岩地貌而著名。你可以在这里漫步世界自然遗产地，欣赏别具一格的自然景观。\n\n除了以上几个地方，云南还有很多其他值得一游的景点，比如西双版纳的热带雨林、腾冲的地热地貌、元阳梯田等等。总之，云南是一个充满魅力的旅行目的地，无论你是喜欢自然风光还是文化探索，都能在这里找到适合自己的旅行体验。'),
 AIMessage(content='上海是中国最具活力和国际化的城市之一，有很多有趣的景点和活动可以参加。一些热门的旅游景点包括：\n\n1. 外滩：外滩是上海的标志性景点，夜晚时分可以欣赏到迷人的灯光秀和城市天际线。\n\n2. 东方明珠塔：这座高达468米的塔楼是上海的地标之一，游客可以乘坐观光电梯俯瞰整个城市。\n\n3. 田子坊：这是一个充满历史和文化气息的老街区，有很多有趣的小店和餐馆。\n\n4. 上海迪士尼度假区：如果你喜欢迪士尼乐园，可以在这里度过一天，享受各种游乐设施和表演。\n\n5. 上海博物馆：如果你对中国历史和文化感兴趣，可以去博物馆参观展品。\n\n此外，上海还有很多美食、购物和夜生活场所，可以让你度过愉快的假期。希望你在上海旅行愉快！')]

## Async Stream

In [15]:
async for s in chain.astream({"topic": "云南"}):
    print(s.content, end="", flush=True)

云南是一个美丽而多样化的地方，拥有丰富的自然风光和文化遗产。以下是一些云南旅行的推荐景点和活动：

1. 丽江古城：这是一个保存完好的古代城市，被列入世界文化遗产名录。游客可以在这里欣赏到古老的建筑和传统的纳西族文化。

2. 玉龙雪山：这座雄伟的雪山是丽江的标志性景点，游客可以乘坐缆车到达山顶，欣赏壮丽的雪景和高山植被。

3. 三江并流：这里是长江、澜沧江和怒江的交汇处，游客可以乘船游览这个壮丽的景点，感受三条江河的气势和美丽。

4. 石林：这个地质奇观是由钟乳石组成的，被誉为“天下石林”的石林景区是世界自然遗产。

5. 云南民族村寨：云南有众多少数民族，他们各自有着独特的风俗和文化。游客可以前往民族村寨，了解不同民族的传统生活和文化。

除了以上景点，云南还有许多户外活动，如徒步旅行、骑行、探险等，适合喜欢探险和户外活动的游客。同时，云南的美食也是一大亮点，有着各种口味丰富的美食，如过桥米线、火腿粑粑等。希望你在云南旅行中能够找到适合自己的玩乐方式，度过一个难忘的旅行。

## Async Invoke

In [16]:
await chain.ainvoke({"topic": "云南"})

AIMessage(content='云南是中国的一个美丽省份，拥有许多令人惊叹的自然景观和丰富的文化遗产。如果你想去云南旅行，以下是一些你可以体验的好玩的活动：\n\n1. 丽江古城：这是一个保存完好的古老城市，被联合国教科文组织列为世界文化遗产。你可以在这里漫步古老的街道，欣赏传统的纳西民居和体验当地的文化和美食。\n\n2. 腾冲地热海：这是中国最大的地热海，拥有许多温泉和独特的地热景观。你可以在这里享受温泉浴，放松身心。\n\n3. 三江并流：这是一个壮美的自然景观，三条河流在这里交汇，形成了壮观的水域景观。你可以乘船游览这个地方，欣赏壮丽的自然风光。\n\n4. 临沧茶马古道：这是一个古老的商业贸易路线，连接云南和西藏。你可以沿着这条古道徒步旅行，体验古老的商业文化和壮丽的自然风光。\n\n5. 石林风景区：这是一个举世闻名的自然景观，拥有许多奇特的石灰岩地貌。你可以在这里漫步观赏这些奇特的石林景观，感受大自然的鬼斧神工。\n\n除了以上提到的景点，云南还有许多其他值得一游的地方，如泸沽湖、西双版纳、玉龙雪山等。无论你是喜欢自然风光还是文化遗产，云南都会给你带来一个难忘的旅行体验。')

## Async Batch

In [18]:
await chain.abatch([{"topic": "云南"}, {"topic": "上海"}])

[AIMessage(content='云南是中国南部的一个美丽省份，拥有丰富的自然景观和文化遗产。如果你想去云南旅行，这里有很多好玩的地方和活动：\n\n1.大理古城：这是一个保存完好的古城，拥有精美的建筑和古老的历史，可以漫步其中感受古老的氛围。\n\n2.丽江古城：丽江古城是世界文化遗产，拥有独特的纳西文化和壮丽的自然景观，是一个很好的旅游目的地。\n\n3.玉龙雪山：这是云南的一个著名景点，可以进行徒步、滑雪和观赏雪山景观等活动。\n\n4.香格里拉：这是一个藏区风情浓厚的地方，有美丽的自然景观和独特的藏传佛教文化。\n\n5.泸沽湖：这是一个高原湖泊，有纯净的湖水和壮丽的山水景观，可以进行划船、徒步等活动。\n\n除此之外，云南还有很多其他的景点和活动，比如虎跳峡、西双版纳、石林等。总的来说，云南是一个充满魅力的旅游目的地，可以满足不同类型旅行者的需求。'),
 AIMessage(content='上海是一个充满活力和魅力的城市，有很多好玩的地方可以去。以下是一些你可以在上海体验的活动和景点：\n\n1. 上海外滩：这是上海最著名的地标之一，这里可以欣赏到城市的壮丽夜景，还可以欣赏到不同风格的建筑。\n\n2. 东方明珠塔：这座世界著名的地标建筑是上海的标志之一，可以乘坐电梯登上塔顶，俯瞰整个上海城市景观。\n\n3. 豫园：这是一个古老的园林，里面有传统的中国建筑和美丽的园林景观，是一个可以放松心情的好去处。\n\n4. 上海迪士尼度假区：如果你喜欢游乐园和迪士尼角色，可以去上海迪士尼度假区体验一下。\n\n5. 上海博物馆：这是一个了解上海历史和文化的好地方，里面有丰富的展品和文物。\n\n6. 复兴岛：这是一个文艺气息浓厚的创意园区，有很多文化艺术活动和展览。\n\n除此之外，上海还有许多美食和购物的好去处，可以尝试当地的特色美食，感受上海的时尚气息。希望你能在上海度过一个愉快的旅行！')]

## Async Stream Intermediate Steps

所有可运行的对象还有一个方法 .astream_log()，用于流式传输（在发生时）您链/序列的所有或部分中间步骤。
这对于向用户显示进度、使用中间结果或调试您的链条非常有用。
您可以流式传输所有步骤（默认），也可以按名称、标签或元数据包含/排除步骤。
该方法产生 JSONPatch 操作，当以接收到的相同顺序应用时构建 RunState。

In [ ]:
from ast import Dict, List
from typing import Optional, TypedDict

from traitlets import Any


class LogEntry(TypedDict):
    id: str
    """ID of the sub-run."""
    name: str
    """Name of the object being run."""
    type: str
    """Type of the object being run, eg. prompt, chain, llm, etc."""
    tags: List[str]
    """List of tags for the run."""
    metadata: Dict[str, Any]
    """Key-value pairs of metadata for the run."""
    start_time: str
    """ISO-8601 timestamp of when the run started."""

    streamed_output_str: List[str]
    """List of LLM tokens streamed by this run, if applicable."""
    final_output: Optional[Any]
    """Final output of this run.
    Only available after the run has finished successfully."""
    end_time: Optional[str]
    """ISO-8601 timestamp of when the run ended.
    Only available after the run has finished."""


class RunState(TypedDict):
    id: str
    """ID of the run."""
    streamed_output: List[Any]
    """List of output chunks streamed by Runnable.stream()"""
    final_output: Optional[Any]
    """Final output of the run, usually the result of aggregating (`+`) streamed_output.
    Only available after the run has finished successfully."""

    logs: Dict[str, LogEntry]
    """Map of run names to sub-runs. If filters were supplied, this list will
    contain only the runs that matched the filters."""

### 流式传输 JSONPatch 块

这很有用，例如在HTTP服务器中流式传输JSONPatch，然后在客户端应用操作以重建运行状态。请参阅LangServe工具，以便更轻松地从任何可运行的程序构建Web服务器。

In [23]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """只回答关于Context里内容的问题。如果在Context内容未找到答案将如实告诉用户：抱歉我还未学习该知识，请谅解。:
Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(
    texts=["2023年亚运会在杭州举行","2022年冬奥会在北京举行","2021年世界杯在卡塔尔举行"], 
    embedding=OpenAIEmbeddings(

    )
)
retriever = vectorstore.as_retriever()

retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

async for chunk in retrieval_chain.astream_log(
    "2023年亚运会在哪举行?", include_names=["Docs"]
):
    print("-" * 40)
    print(chunk)

----------------------------------------
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '4a9f7d5d-4155-49b6-bb65-0c3e044ce7e4',
            'logs': {},
            'streamed_output': []}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs',
  'value': {'end_time': None,
            'final_output': None,
            'id': '4a50b948-841f-45ac-ba2f-dc5804d77ce4',
            'metadata': {},
            'name': 'Docs',
            'start_time': '2023-12-22T05:50:25.463',
            'streamed_output_str': [],
            'tags': ['map:key:context', 'FAISS', 'OpenAIEmbeddings'],
            'type': 'retriever'}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs/final_output',
  'value': {'documents': [Document(page_content='2023年亚运会在杭州举行'),
                          Document(page_content='2022年冬奥会在北京举行'),
                          Document(page_content='2021年世界杯在卡塔

# Streaming the incremental RunState 
您可以简单地将**diff=False**传递给获取RunState的增量值。通过更多重复部分，您可以获得更详细的输出。

In [24]:
async for chunk in retrieval_chain.astream_log(
    "2023年亚运会在哪举行?", include_names=["Docs"],diff=False
):
    print("-" * 40)
    print(chunk)

----------------------------------------
RunLog({'final_output': None,
 'id': '40f7d769-2e3c-48b2-96b0-4888fb111636',
 'logs': {},
 'streamed_output': []})
----------------------------------------
RunLog({'final_output': None,
 'id': '40f7d769-2e3c-48b2-96b0-4888fb111636',
 'logs': {'Docs': {'end_time': None,
                   'final_output': None,
                   'id': 'c587076b-f508-41d9-b323-0168c88926d2',
                   'metadata': {},
                   'name': 'Docs',
                   'start_time': '2023-12-22T06:12:19.901',
                   'streamed_output_str': [],
                   'tags': ['map:key:context', 'FAISS', 'OpenAIEmbeddings'],
                   'type': 'retriever'}},
 'streamed_output': []})
----------------------------------------
RunLog({'final_output': None,
 'id': '40f7d769-2e3c-48b2-96b0-4888fb111636',
 'logs': {'Docs': {'end_time': '2023-12-22T06:12:22.395',
                   'final_output': {'documents': [Document(page_content='2023年亚运会在杭州举行'

## Parallelism
让我们来看一下LangChain表达语言如何支持并行请求。例如，当使用RunnableParallel（通常写成字典）时，它会并行执行每个元素。

In [25]:
from langchain_core.runnables import RunnableParallel

chain1 = ChatPromptTemplate.from_template("告诉我 {topic}这里在什么地方！") | model
chain2 = (
    ChatPromptTemplate.from_template("这里有什么好玩的 {topic}")
    | model
)
combined = RunnableParallel(where=chain1, play=chain2)


In [27]:
%%time
chain1.invoke({"topic": "上海"})

CPU times: total: 0 ns
Wall time: 11.2 s


AIMessage(content='上海位于中国的东部沿海地区，地处长江入海口附近，是中国最大的城市之一，也是国际金融、贸易中心。上海市的地理位置是北纬31°14′～31°53′，东经121°21′～122°12′。')

In [28]:
%%time
chain2.invoke({"topic": "上海"})

CPU times: total: 0 ns
Wall time: 7.59 s


AIMessage(content='上海有很多好玩的地方，比如外滩，可以欣赏到上海的美丽夜景；豫园，可以感受到古老的园林风情；迪士尼乐园，可以体验到迪士尼的欢乐和刺激；田子坊，可以感受到上海特有的小资情调；上海科技馆，可以了解到科技的最新发展。还有很多其他的景点和活动，可以根据自己的兴趣选择。')

In [30]:
%%time
combined.invoke({"topic": "上海"})

CPU times: total: 15.6 ms
Wall time: 14.4 s


{'where': AIMessage(content='上海位于中国的东部沿海地区，是中国第一大城市，也是全国经济、金融、贸易、航运中心。具体来说，上海位于中国的东部沿海地区，东临东海，南临浙江省，西、北、北部与江苏省毗邻。'),
 'play': AIMessage(content='上海有很多好玩的地方，例如：\n\n1. 上海迪士尼乐园：是亚洲第一个迪士尼乐园，拥有丰富多彩的游乐设施和精彩的演出。\n\n2. 外滩：是上海的标志性景点，夜晚时分可以欣赏到灯火辉煌的城市夜景。\n\n3. 东方明珠电视塔：是上海的地标建筑之一，可以乘坐观光电梯俯瞰整个上海市区。\n\n4. 田子坊：是一个充满文艺气息的老街区，有很多小店、咖啡馆和艺术工作室。\n\n5. 上海海洋水族馆：是亚洲最大的海洋馆之一，有各种海洋动物和表演，适合全家游玩。\n\n6. 上海动物园：有丰富的动物种类和精彩的表演，是孩子们喜欢的地方。\n\n除此之外，上海还有很多购物中心、美食街和艺术展览等各种娱乐活动，适合不同年龄段的游客。')}

Parallelism on batches（批量并行执行）

In [31]:
%%time
chain1.batch([{"topic": "上海"}, {"topic": "云南"}])

CPU times: total: 15.6 ms
Wall time: 5.81 s


[AIMessage(content='上海位于中国的东部沿海地区，是中国最大的城市之一，也是全球最大的城市之一。它位于长江口附近，东临东海，是中国重要的经济、金融和文化中心。上海市的地理坐标为北纬31°14′至31°53′，东经121°29′至122°20′。'),
 AIMessage(content='云南位于中国的西南部，毗邻四川、贵州、广西和西藏，与缅甸、老挝、越南相邻。云南省的省会是昆明市。')]

In [32]:
%%time
chain2.batch([{"topic": "上海"}, {"topic": "云南"}])

CPU times: total: 0 ns
Wall time: 14 s


[AIMessage(content='上海是一个充满活力和魅力的城市，有很多好玩的地方可以去。\n\n1. 外滩：夜晚的外滩灯火辉煌，是上海最具代表性的景点之一，可以欣赏到浦东的夜景和黄浦江的美丽。\n\n2. 东方明珠塔：作为上海的标志性建筑之一，东方明珠塔是上海的地标之一，登塔俯瞰整个城市的景色壮观。\n\n3. 田子坊：这里是上海最具有历史和文化气息的地方，可以感受到老上海的风情，有很多小店和餐厅可以逛逛品尝。\n\n4. 上海迪士尼乐园：对于喜欢游乐园的游客来说，上海迪士尼乐园是一个绝佳的选择，可以和家人朋友一起度过愉快的时光。\n\n5. 复兴岛：这里是上海的创意产业聚集地，有很多时尚的酒吧、咖啡馆和艺术展览可以参观。\n\n总的来说，上海是一个充满活力和多姿多彩的城市，有很多好玩的地方可以让人留连忘返。'),
 AIMessage(content='云南是一个充满活力和多样文化的地方，有许多好玩的活动和景点可以体验。以下是一些云南的好玩之处：\n\n1. 丽江古城：这个古老的城市保留着许多传统的建筑和文化，是一个迷人的地方，可以漫步在石板路上，欣赏古老的建筑和品尝当地美食。\n\n2. 玉龙雪山：这座雪山是云南著名的旅游胜地，可以搭乘缆车到达山顶，欣赏壮观的雪景。\n\n3. 东巴文化村：这个村落保留着东巴文化的传统，可以了解当地独特的宗教和文化。\n\n4. 香格里拉：这个地方被誉为“人间天堂”，有着壮丽的自然风光和丰富的藏传佛教文化。\n\n5. 云南民族村：在这里可以了解云南各个少数民族的传统文化和生活方式，体验当地的民俗活动。\n\n总的来说，云南有丰富的自然风光和多样的文化，游客可以在这里体验到不同的乐趣和挑战。')]

In [34]:
%%time
combined.batch([{"topic": "上海"}, {"topic": "云南"}])

CPU times: total: 31.2 ms
Wall time: 18.7 s


[{'where': AIMessage(content='上海位于中国的东部沿海地区，地处长江口，是中国最大的城市之一，也是全国重要的经济、金融、贸易和航运中心。上海市地理位置为北纬30° 40′ — 31° 53′，东经120° 51′ — 122° 12′。'),
  'play': AIMessage(content='上海是一个充满活力和魅力的城市，有很多好玩的地方可以探索。\n\n1. 上海迪士尼乐园：作为中国大陆第一个迪士尼乐园，这里有各种刺激的游乐设施和迪士尼经典角色的表演，是家庭和朋友一起玩乐的好去处。\n\n2. 外滩：这是上海最具代表性的景点之一，可以欣赏到黄浦江两岸的现代建筑和历史建筑，尤其是在夜晚灯火辉煌的时候，景色格外迷人。\n\n3. 东方明珠电视塔：这座标志性建筑是上海的地标之一，可以乘坐观光电梯上到观景台，俯瞰整个上海城市的美景。\n\n4. 豫园：这是一座古老的园林，有精美的园林景观和古建筑，可以感受到传统中国园林的韵味。\n\n5. 复兴岛：这个新兴的文化创意产业区充满了各种时尚的店铺、美食餐厅和艺术展览，是年轻人喜欢的打卡地点。\n\n除此之外，上海还有许多博物馆、艺术馆、购物中心、美食街等等，可以满足各种不同兴趣爱好的游客。总之，上海是一个充满活力和魅力的城市，有很多好玩的地方等待着你去发现。')},
 {'where': AIMessage(content='云南位于中国的西南部，毗邻缅甸、老挝和越南。'),
  'play': AIMessage(content='云南是一个充满活力和多样文化的地方，有许多有趣的活动和景点可以探索。一些好玩的活动包括:\n\n1. 丽江古城：可以漫步在古老的街道上，欣赏传统的建筑和文化表演。\n\n2. 玉龙雪山：可以乘坐缆车登上雪山，欣赏壮丽的雪景和高山美景。\n\n3. 东巴石刻：这些古老的石刻记录了东巴文化的历史和传统，非常值得一看。\n\n4. 西双版纳热带雨林：可以参加热带雨林的探险之旅，体验大自然的奇妙和多样性。\n\n5. 云南民族村寨：可以了解和体验不同少数民族的传统文化和生活方式。\n\n此外，云南还有许多美丽的自然风景和特色美食等等，让人流连忘返。')}]